In [ ]:
using System;
using System.IO;
using System.Collections.Generic;
using System.Collections;
using System.Threading;
using System.Text.Json;

In [ ]:
public class Debug : IDisposable
{
    public static bool IsEnabled {get; private set;} = false;

    public static void Print(string input)
    {
        if(IsEnabled)
        {
            Console.WriteLine(input);
        }
    }

    public Debug(bool enable = true)
    {
        IsEnabled = enable;
    }

    public void Dispose()
    {
        IsEnabled = false;
    }
}

In [ ]:
var input = File.ReadAllLines("sample.txt");

In [ ]:
public static class Mappings
{
    public static int i = 0;
    public static string Fake()
    {
        return $"X{i++}";
    }
}

In [ ]:
public record Replacement
{
    public string From {get; init;}
    public string To {get; init;}
    public bool IsReal {get; init;}
}

In [ ]:
public static IEnumerable<string> Splice(string input)
{
    for(int i = 0; i < input.Length; i++)
    {        
        if(i < input.Length - 1 && char.IsLower(input[i + 1]))
        {
            yield return string.Concat(input[i], input[i + 1]);
            i++;
        }
        else{
            yield return string.Concat(input[i]);
        }
    }
}

In [ ]:
public IEnumerable<Replacement> Parse(string input)
{
    var segments = input.Split("=>", 2, StringSplitOptions.TrimEntries);
    var count = segments[1].Count(c => char.IsUpper(c));
    
    var list = Splice(segments[1]).ToArray();
    
    if(list.Count() == 1)
    {
        yield return new Replacement
        {
            From = segments[0],
            To = segments[1],
            IsReal = true
        };

        yield break;
    }

    var from = "";
    var to = list[0] + list[1];

    for(int i = 2; i < count; i++)
    {
        from = Mappings.Fake();

        yield return new Replacement
        {
            From = from,
            To = to,
            IsReal = false
        };

        to = from + list[i];
    }

    from = segments[0];

    yield return new Replacement
    {
        From = from,
        To = to,
        IsReal = true
    };

    yield break;
}

In [ ]:
var replacements = new List<Replacement>();
string target;

foreach(var line in input)
{
    if(line.Contains("=>"))
    {
        replacements.AddRange(Parse(line));
    }
    else if(!string.IsNullOrWhiteSpace(line))
    {
        target = line;
    }
}

In [ ]:
replacements.GroupBy(c => c.To)

index,value
0,"[ { Replacement { From = Al, To = ThF, IsReal = True }: From: Al, To: ThF, IsReal: True } ]"
1,"[ { Replacement { From = X0, To = ThRn, IsReal = False }: From: X0, To: ThRn, IsReal: False } ]"
2,"[ { Replacement { From = X1, To = X0F, IsReal = False }: From: X1, To: X0F, IsReal: False } ]"
3,"[ { Replacement { From = Al, To = X1Ar, IsReal = True }: From: Al, To: X1Ar, IsReal: True } ]"
4,"[ { Replacement { From = B, To = BCa, IsReal = True }: From: B, To: BCa, IsReal: True } ]"
5,"[ { Replacement { From = B, To = TiB, IsReal = True }: From: B, To: TiB, IsReal: True } ]"
6,"[ { Replacement { From = X2, To = TiRn, IsReal = False }: From: X2, To: TiRn, IsReal: False } ]"
7,"[ { Replacement { From = X3, To = X2F, IsReal = False }: From: X3, To: X2F, IsReal: False } ]"
8,"[ { Replacement { From = B, To = X3Ar, IsReal = True }: From: B, To: X3Ar, IsReal: True } ]"
9,"[ { Replacement { From = Ca, To = CaCa, IsReal = True }: From: Ca, To: CaCa, IsReal: True } ]"


In [ ]:
public record Match
{
    public int Count {get; set;}
    public string S {get; init;}
}

In [ ]:
var s = Splice(target).ToArray();

var count = s.Count();
var layers = new List<Match>[count][];
for(int i = 0; i < count; i++)
{
    layers[i] = new List<Match>[count - i];

    for(int n = 0; n < count - i; n++)
    {
        layers[i][n] = new List<Match>();
    }
}

for(int i = 0; i < count; i++)
{
    layers[0][i].Add(new Match{S = s[i], Count = 0});
}

In [ ]:
for(int i = 1; i < count; i++)
{
    for(int n = 0; n < count - i; n++)
    {
        // Console.WriteLine($"[{i}][{n}]");
        var current = layers[i][n];

        for(int o = 0; o < i; o++)
        {
            List<Match> left = null, right = null;

            if(i == 1)
            {
                left = layers[0][n];
                right = layers[0][n + 1];
            }
            else{
                left = layers[o][n];
                right = layers[i - 1 - o][n + 1 + o];
            }

            foreach(var l in left)
            {
                foreach(var r in right)
                {
                    var m = l.S + r.S;
                    var candidates = replacements.Where(c => c.To == m);
                    foreach(var replacement in candidates)
                    {
                        var match = new Match{S = replacement.From, Count = l.Count + r.Count + 1};                    
                        var existing = current.FirstOrDefault(c => c.S == match.S);

                        if(existing != null)
                        {
                            existing.Count = Math.Min(existing.Count, match.Count);
                        }
                        else{
                            current.Add(match);
                        }
                        
                        // Console.WriteLine($"{replacement.From} => {replacement.To}");
                    }
                }
            }
        }
    }
}

In [ ]:
layers.Last()

index,value
0,"[ { Match { Count = 273, S = e }: Count: 273, S: e } ]"


In [ ]:
// foreach(var n in layers.Select(d => d.Count(c => c.Any())).Reverse())
// {
//     Console.WriteLine(n);
// }

In [ ]:
public void Print(List<Match>[][] layers)
{
    foreach(var layer in layers)
    {
        foreach(var line in layer)
        {
            Console.Write("|");
            Console.Write(string.Join(';', line.Select(c => c.S)));
            Console.Write("|\t");
        }
        Console.WriteLine();
    }
}

In [ ]:
// Print(layers)